# **Libs**

In [ ]:
# Torch
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms, models

# Utils
from src.utils import *

# **Code**

## Зафиксируем seed

In [ ]:
set_all_seeds()

## Data

### Reading

In [ ]:
path = r'../data/train'

dataset, classes = reading_dataset_from_folders(path)

In [ ]:
# df = pd.read_csv("../data/train.csv")
# path = r'../data/train'

# dataset = reading_dataset_from_file(path, df.set_index('image_name')['class_id'])
# classes = df.drop_duplicates('class_id').sort_values('class_id')['unified_class'].tolist()

### Splitting

In [ ]:
trainset, validset = split_dataset(dataset, 0.2) # Разбиваем dataset на trainset, validset

### **Transformation** and **augmentation**

In [ ]:
# Определим базовые преобразования 
transform = transforms.Compose([
    transforms.ToTensor(),                      # Преобразовать в тензор
    transforms.Normalize(mean=mean, std=std)    # Нормализовать данные
])

# Определим преобразования с аугментациями
augmentation = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),                                         # Случайное горизонтальное отражение
    transforms.RandomVerticalFlip(p=0.5),                                           # Случайное вертикальное отражение
    transforms.RandomRotation(degrees=15),                                          # Случайный поворот на ±15 градусов
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),                       # Случайное смещение
    transforms.RandomResizedCrop(size=image_size[1], scale=(0.8, 1.0)),             # Случайное кадрирование и изменение размера
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Случайное изменение цветов
    transforms.RandomGrayscale(p=0.2),                                              # Случайное преобразование в оттенки серого
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),                      # Случайная перспектива
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3)),           # Случайное удаление части изображения
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),                       # Случайное размытие
    transforms.ToTensor(),                                                          # Преобразовать в тензор
    transforms.Normalize(mean=mean, std=std)                                        # Нормализовать данные
])

### Processing

In [ ]:
*trainsets, validset = ( # обработка данных
    list(map(lambda x: (t(x[0]), x[1]), tqdm(d)))
    for d, t in [
        (trainset, transform),
        (trainset, augmentation),
        (validset, transform),
    ]
)

dataset = trainsets[0] + validset

trainset = list()
for temp in trainsets:
    trainset += temp

### Visualization

In [ ]:
show_image(dataset, classes)

### Creating a DataLoader

In [ ]:
# Создание DataLoader для каждой выборки
batch_size = 24

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
validloader = DataLoader(validset, batch_size=batch_size, shuffle=True)

## Models

### Score

In [ ]:
scores = dict()

In [ ]:
def f1_macro(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

### MyModel

In [ ]:
# Сверточная нейронная сеть
mymodel = nn.Sequential(
    Conv2dBlock(image_size[0], 9),
    Conv2dBlock(9, 16),
    SkipConnection(
        Conv2dBlock(16, 32, False),
        Conv2dBlock(32, 16, False)
    ),
    Conv2dBlock(16, 32),
    Conv2dBlock(32, 64),
    nn.Flatten(),
    LinearBlock(12544, 1500),
    SkipConnection(
        LinearBlock(1500, 1500)
    ),
    LinearBlock(1500, 500),
    LinearBlock(500, len(classes))
)

mymodel = Model(mymodel)

In [ ]:
optimizer = optim.Adam(mymodel.parameters(), lr=3e-4)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
mymodel.fit(trainloader, validloader, optimizer, loss_fn, 15, f1_macro, 'MyModel', min_loss=True)
scores[mymodel.best_score] = mymodel

In [ ]:
# mymodel.load("models/mymodel.pth")

### ResNet

In [ ]:
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)  
resnet.fc = nn.Linear(resnet.fc.in_features, len(classes))  

resnet = Model(resnet)

In [ ]:
optimizer = optim.Adam(resnet.parameters(), lr=3e-4)  
loss_fn = nn.CrossEntropyLoss()

In [ ]:
resnet.fit(trainloader, validloader, optimizer, loss_fn, 15, f1_macro, 'ResNet', min_loss=True)  
scores[resnet.best_score] = resnet

In [ ]:
# resnet.load("models/resnet.pth")

### VGG

In [ ]:
vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)  
vgg.classifier[6] = nn.Linear(vgg.classifier[6].in_features, len(classes))

vgg = Model(vgg)

In [ ]:
optimizer = optim.Adam(vgg.parameters(), lr=3e-4)  
loss_fn = nn.CrossEntropyLoss()

In [ ]:
vgg.fit(trainloader, validloader, optimizer, loss_fn, 15, f1_macro, 'VGG', min_loss=True)  
scores[vgg.best_score] = vgg

In [ ]:
# vgg.load("models/vgg.pth")

### EfficientNet

In [ ]:
efficientnet = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
efficientnet.classifier[1] = nn.Linear(efficientnet.classifier[1].in_features, len(classes))

efficientnet = Model(efficientnet)

In [ ]:
optimizer = optim.Adam(efficientnet.parameters(), lr=3e-4)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
efficientnet.fit(trainloader, validloader, optimizer, loss_fn, 15, f1_macro, 'EfficientNet', min_loss=True)
scores[efficientnet.best_score] = efficientnet

In [ ]:
# efficientnet.load("models/efficientnet.pth")

### DenseNet-201

In [ ]:
densenet = models.densenet201(weights=models.DenseNet201_Weights.IMAGENET1K_V1)
densenet.classifier = nn.Linear(densenet.classifier.in_features, len(classes))

densenet = Model(densenet)

In [ ]:
optimizer = optim.Adam(densenet.parameters(), lr=3e-4)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
densenet.fit(trainloader, validloader, optimizer, loss_fn, 15, f1_macro, 'DenseNet', min_loss=True)
scores[densenet.best_score] = densenet

In [ ]:
# densenet.load("models/densenet.pth")

### MobileNet

In [ ]:
mobilenet = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V1)  
mobilenet.classifier[3] = nn.Linear(mobilenet.classifier[3].in_features, len(classes))  

mobilenet = Model(mobilenet)

In [ ]:
optimizer = optim.Adam(mobilenet.parameters(), lr=3e-4)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
mobilenet.fit(trainloader, validloader, optimizer, loss_fn, 15, f1_macro, 'MobileNet', min_loss=True)
scores[mobilenet.best_score] = mobilenet

In [ ]:
# mobilenet.load("models/mobilenet.pth")

## Result

In [ ]:
best_model = scores[max(scores)] # ? Выбрать модель

In [ ]:
print('Calculation:')
loss, score = best_model.evaluate(validloader, loss_fn, metrics)
print(f"\nLoss: {loss:.4f}\n")
print("Scores:")
print(score)

## Submission

In [ ]:
path = r'../data/test'

testset, image_names = reading_testset(path, transform=transform)

In [ ]:
predict = best_model.predict(testset)